In [1]:
import import_ipynb

from no_signalling_sampler import *

In [8]:
local = sample_no_signalling(10000, True)
non_local = sample_no_signalling(10000, False)

In [7]:
from sklearn.dataset import train_test_split
X_train, y_train, X_test, y_test = train_test_split()

array([[0.18091036, 0.34708906, 0.13562979, 0.39236963],
       [0.37136926, 0.10063132, 0.25113257, 0.22086801],
       [0.21972077, 0.06365915, 0.00633073, 0.27704919],
       [0.33255885, 0.38406124, 0.38043163, 0.33618845]])

In [18]:
!pip install pytorch-lightning
  

     |████████████████████████████████| 523 kB 818 kB/s eta 0:00:01
     |████████████████████████████████| 5.8 MB 2.6 MB/s eta 0:00:01
     |████████████████████████████████| 701 kB 1.8 MB/s eta 0:00:01
Processing /home/johnny/.cache/pip/wheels/8e/70/28/3d6ccd6e315f65f245da085482a2e1c7d14b90b30f239e2cf4/future-0.18.2-py3-none-any.whl
     |████████████████████████████████| 329 kB 1.8 MB/s eta 0:00:01
  Using cached tqdm-4.62.3-py2.py3-none-any.whl (76 kB)
     |████████████████████████████████| 132 kB 2.2 MB/s eta 0:00:01
  Using cached google_auth_oauthlib-0.4.6-py2.py3-none-any.whl (18 kB)
     |████████████████████████████████| 155 kB 2.8 MB/s eta 0:00:01
  Using cached tensorboard_plugin_wit-1.8.0-py3-none-any.whl (781 kB)
     |████████████████████████████████| 126 kB 1.6 MB/s eta 0:00:01
     |████████████████████████████████| 4.0 MB 2.3 MB/s eta 0:00:01
     |████████████████████████████████| 97 kB 2.9 MB/s eta 0:00:01
  Using cached tensorboard_data_server-0.6.1-py3-none-manyl

In [15]:
import pytorch_lightning as pl

class LITAutoEncoder(pl.LightningModule):
    def __init__(self):
        super().__init__()
        self.model = nn.Sequential(n.Linear(28 * 28, 64), n.ReLU(), nn.Linear(64, 3))

    def forward(self, x):
        return self.model(x)
    
    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters, lr=1e-3)
        return optimizer

    def training_step(self, train_batch, batch_idx):
        x, y = train_batch
        y_hat = self.model(x)
        loss = F.binary_cross_entropy(y_hat, y)
        self.log(‘train_loss’, loss)
        return loss

    def validation_step(self, val_batch, batch_idx):
        x, y = val_batch
        x = x.view(x.size(0), -1)
        y_hat = self.model(x)
        loss = F.binary_cross_entropy(y_hat, y)
        val_loss.append(loss)
        self.log('val_loss', loss)

SyntaxError: invalid character in identifier (2097247901.py, line 19)